In [1]:
import basic_image_operations as basic_ops
import filters
import geometrical_image_operations as geo_ops
import traffic_sign_detection as tsd
import colors
import numpy as np
import statistical_operations as stat_ops

'''stop_img = basic_ops.load_image(image_path='traffic_sign_templates/stop_signs/stop.jpg')
vf_img = basic_ops.load_image(image_path='traffic_sign_templates/vf_signs/vf.jpg')
vfa_img = basic_ops.load_image(image_path='traffic_sign_templates/vfa_signs/vfa.jpg')
vfs_img = basic_ops.load_image(image_path='traffic_sign_templates/vfs_signs/vfs.jpg')'''
#images = basic_ops.load_images(folder_path='data', amount=3)
images = [basic_ops.load_image(image_path='traffic_sign_images/vfs_01.jpg')]
color_functions = [colors.is_strong_red, colors.is_strong_yellow, colors.is_strong_blue]
images = [geo_ops.resize_image(image=image, target_height=image.shape[0]//8, target_width=image.shape[1]//8) for image in images]

Image loaded from traffic_sign_images/vfs_01.jpg


In [2]:
for i, image in enumerate(images):
    image = filters.median_filter(image=image, dim=3)
    image = stat_ops.gamma_equalization(image=image, gamma=0.25)
    images[i] = image

In [3]:
all_bounding_boxes = []
for i, image in enumerate(images):
    image_bounding_boxes = []
    img = np.copy(image)
    for color_function in color_functions:
        mask = colors.get_mask(image=img, color_function=color_function)
        blobs = tsd.get_blobs(mask=mask)
        img, bounding_boxes = tsd.draw_bounding_boxes(image=img, blobs=blobs, min_box_area=200, box_color=colors.get_color_from_function(color_function))
        image_bounding_boxes.extend(bounding_boxes)
    all_bounding_boxes.extend(image_bounding_boxes)
    print(all_bounding_boxes)
    images[i] = img

[BoundingBox(center_y=199, center_x=76, height=18, width=24, area=432, label='y'), BoundingBox(center_y=242, center_x=69, height=27, width=40, area=1080, label='y'), BoundingBox(center_y=256, center_x=168, height=38, width=35, area=1330, label='y')]


In [4]:
for i, image in enumerate(images):
    p2, p98 = np.percentile(image, (2, 98))
    image = np.clip((image - p2) * 255.0 / (p98 - p2), 0, 255).astype(np.uint8)
    images[i] = image
    basic_ops.show_image(image=image)

Image displayed
